<a href="https://colab.research.google.com/github/akpanitorobong/7135CEM-Modelling_and_Optimization_Under_Uncertainty/blob/main/7135CEM_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Document Preparation**

**Necessary Imports**

In [4]:
print("Importing necessary libraries...")
!pip install ucimlrepo

from ucimlrepo import fetch_ucirepo # For dataset import
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical computations
import seaborn as sns  # For data visualization
import matplotlib.pyplot as plt  # For plotting graphs

from sklearn.model_selection import train_test_split, cross_val_score  # For splitting data and cross-validation
from sklearn.preprocessing import StandardScaler, LabelEncoder  # For feature scaling and encoding categorical variables
from sklearn.ensemble import RandomForestClassifier  # Random Forest model for classification
from sklearn.linear_model import LogisticRegression  # Logistic Regression model for baseline comparison
from sklearn.gaussian_process import GaussianProcessClassifier  # Gaussian Process Classification model
from sklearn.gaussian_process.kernels import RBF, Matern  # RBF and Matérn kernels for Gaussian Process
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix  # Evaluation metrics
print("Libraries imported successfully.")

**Get Datasets**

In [12]:
print("Loading dataset...")
# fetch dataset
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# data (as pandas dataframes)
X = cdc_diabetes_health_indicators.data.features
y = cdc_diabetes_health_indicators.data.targets
df = pd.concat([X, y], axis=1)
print("Dataset loaded successfully.")
df.head()

Loading dataset...
Dataset loaded successfully.


,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,1,1,1,40,1,0,0,0,0,1,...,0,5,18,15,1,0,9,4,3,0
1,0,0,0,25,1,0,0,1,0,0,...,1,3,0,0,0,0,7,6,1,0
2,1,1,1,28,0,0,0,0,1,0,...,1,5,30,30,1,0,9,4,8,0
3,1,0,1,27,0,0,0,1,1,1,...,0,2,0,0,0,0,11,3,6,0
4,1,1,1,24,0,0,0,1,1,1,...,0,2,3,0,0,0,11,5,4,0


**EDA**

In [11]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())



Missing values:
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
Diabetes_binary         0
dtype: int64
